## NoSQL (MongoDB) (sesión 2)


## Introducción

![MongoDB](https://webassets.mongodb.com/_com_assets/cms/MongoDB_Logo_FullColorBlack_RGB-4td3yuxzjs.png)

Esta hoja muestra cómo acceder a bases de datos MongoDB y también a conectar la salida con Jupyter. Se puede utilizar el *shell* propio de MongoDB en el contenedor usando el programa `mongo`. La diferencia es que ese programa espera código Javascript y aquí trabajaremos con Python.

In [1]:
RunningInCOLAB: bool = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False

In [2]:
if not RunningInCOLAB:
    !sudo apt-get update -qq
    !sudo apt-get install -y -qq gpg p7zip

## Instalación inicial de MongoDB (sólo necesaria en Google Colab)

In [3]:
if RunningInCOLAB:
  !wget -q https://raw.githubusercontent.com/dsevilla/bdge/refs/heads/24-25/addendum/mongo-utils/run-mongo-local.sh
  !bash run-mongo-local.sh

Adding system user `mongodb' (UID 104) ...
Adding new user `mongodb' (UID 104) with group `nogroup' ...
Not creating home directory `/home/mongodb'.
Adding group `mongodb' (GID 107) ...
Done.
Adding user `mongodb' to group `mongodb' ...
Adding user mongodb to group mongodb
Done.
deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/8.0 multiverse
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 10.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-

## Descarga de los datos en formato CSV

 - Formato: 7zipped
 - Ficheros:
   - **Comments**.csv
       - Id
       - PostId
       - Score
       - Text, e.g.: "@Stu Thompson: Seems possible to me - why not try it?"
       - CreationDate, e.g.:"2008-09-06T08:07:10.730"
       - UserId
   - **Posts**.csv
       - Id
       - PostTypeId
          - 1: Question
          - 2: Answer
       - ParentID (only present if PostTypeId is 2)
       - AcceptedAnswerId (only present if PostTypeId is 1)
       - CreationDate
       - Score
       - ViewCount
       - Body
       - OwnerUserId
       - LastEditorUserId
       - LastEditorDisplayName="Jeff Atwood"
       - LastEditDate="2009-03-05T22:28:34.823"
       - LastActivityDate="2009-03-11T12:51:01.480"
       - CommunityOwnedDate="2009-03-11T12:51:01.480"
       - ClosedDate="2009-03-11T12:51:01.480"
       - Title=
       - Tags=
       - AnswerCount
       - CommentCount
       - FavoriteCount
   - **Tags**.csv
    - Id
    - Count
    - ExcerptPostId
    - TagName
    - WikiPostId
   - **Users**.csv
     - Id
     - Reputation
     - CreationDate
     - DisplayName
     - EmailHash
     - LastAccessDate
     - WebsiteUrl
     - Location
     - Age
     - AboutMe
     - Views
     - UpVotes
     - DownVotes
   - **Votes**.csv
     - Id
     - PostId
     - VoteTypeId
        - ` 1`: AcceptedByOriginator
        - ` 2`: UpMod
        - ` 3`: DownMod
        - ` 4`: Offensive
        - ` 5`: Favorite - if VoteTypeId = 5 UserId will be populated
        - ` 6`: Close
        - ` 7`: Reopen
        - ` 8`: BountyStart
        - ` 9`: BountyClose
        - `10`: Deletion
        - `11`: Undeletion
        - `12`: Spam
        - `13`: InformModerator
     - CreationDate
     - UserId (only for VoteTypeId 5)
     - BountyAmount (only for VoteTypeId 9)

In [4]:
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.csv.7z.001 -O - > es.stackoverflow.csv.7z
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.csv.7z.002 -O - >> es.stackoverflow.csv.7z

--2025-02-27 10:23:32--  https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.csv.7z.001
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dsevilla/bd2-data/main/es.stackoverflow/es.stackoverflow.csv.7z.001 [following]
--2025-02-27 10:23:32--  https://raw.githubusercontent.com/dsevilla/bd2-data/main/es.stackoverflow/es.stackoverflow.csv.7z.001
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104857600 (100M) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[===================>] 100.00M   284MB/s    in 0.4s    

2025-02-27 10:23:32 (284 MB/s) - writ

In [5]:
!7zr x -aoa es.stackoverflow.csv.7z
!rm es.stackoverflow.csv.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs AMD EPYC 7B12 (830F10),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 200457538 bytes (192 MiB)

Extracting archive: es.stackoverflow.csv.7z
--
Path = es.stackoverflow.csv.7z
Type = 7z
Physical Size = 200457538
Headers Size = 248
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      1% - Comments.csv                     3% - Comments.csv                     4% - Comments.csv                     6% - Comments.csv                     7% - Comments.csv                     8% - Comments.csv                     9% - Comments.csv                   

In [6]:
!head Users.csv

Id,AboutMe,AccountId,CreationDate,DisplayName,DownVotes,LastAccessDate,Location,Reputation,UpVotes,Views,WebsiteUrl
-1,"<p>Hola, no soy una persona real.</p><br/><br/><p>¡Soy un proceso que ayuda a mantener el sitio limpio!</p><br/><br/><p>Hago cosas como:</p><br/><br/><ul><br/><li>Dar empujoncitos a preguntas antiguas sin respuesta aproximadamente cada hora, para que atraigan algo de atención.</li><br/><li>Tener la propiedad de las preguntas y respuestas wiki para que nadie se lleve reputación por ellas</li><br/><li>Recibir la propiedad de los votos negativos en las publicaciones de spam o dañinas que son borradas permanentemente</li><br/><li>Tener la propiedad de las ediciones sugeridas por usuarios anónimos</li><br/><li><a href=""http://meta.stackoverflow.com/a/92006"">Quitar preguntas abandonadas</a></li><br/></ul><br/>",-1,2015-10-26T21:36:24.767,Comunidad,22504,2015-10-26T21:36:24.767,en la granja de servidores,1,10211,2516,
1,"<p>Dev #2 who helped create Stack Overflow currently

## Instalación de la librería `pymongo`

In [7]:
%pip install pymongo tqdm[notebook] pandas matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.2 MB/s eta 0:00:00


In [8]:
from pprint import pprint as pp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot')

Usaremos la librería `pymongo` para python. La cargamos a continuación.

In [9]:
from pymongo import MongoClient

## Conexión a la base de datos

La conexión se inicia con `MongoClient` en el `host` descrito en el fichero `docker-compose.yml` (`mongo`), o bien a `localhost` si lo estamos haciendo en Colab.

In [10]:
import os
db_hostname: str = "localhost" if RunningInCOLAB else os.getenv('DB_HOSTNAME', "mongo")

In [11]:
from typing import Any

client: MongoClient[dict[str, Any]] = MongoClient(db_hostname,27017)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [12]:
client.list_database_names()

['admin', 'config', 'local']

Las bases de datos se crean conforme se nombran. Se puede utilizar la notación punto o la de diccionario. Las colecciones también.

In [13]:
from pymongo.database import Database

db: Database = client.stackoverflow
# db: Database = client['stackoverflow'] # (equivalente)
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'stackoverflow')

## Importación de los ficheros CSV

Por ahora creamos una colección diferente para cada uno. Después estudiaremos cómo poder optimizar el acceso usando agregación.

In [14]:
import csv
import sys
from datetime import datetime
from pymongo.collection import Collection
from typing import Any
from collections.abc import Callable, Iterable, Iterator
from tqdm.notebook import tqdm

def batched(iterable: Iterable, n) -> Iterable:
    from itertools import islice
    if n < 1:
        raise ValueError('n must be at least one')
    it: Iterator = iter(iterable)
    while batch := tuple(islice(it, n)):
        yield batch

def csv_to_mongo(file: str, coll: Collection):
    """
    Carga un fichero CSV en Mongo. file especifica el fichero, coll la colección
    dentro de la base de datos, y date_cols las columnas que serán interpretadas
    como fechas.
    """
    # Convertir todos los elementos que se puedan a números
    def to_numeric(d: str) -> str| int | float:
        if len(d) == 0:
            return ''
        if not ((d[0] >= '0' and d[0] <= '9') or d[0] == '-' or d[0] == '+' or d[0]=='.'):
            return d
        try:
            return int(d) if abs(int(d)) <= sys.maxsize else d # Ensure number is inside MongoDB max number range
        except ValueError:
            try:
                return float(d)
            except ValueError:
                return d

    def to_date(d: str) -> datetime | None:
        """To ISO Date. If this cannot be converted, return NULL (None)"""
        try:
            return datetime.strptime(d, "%Y-%m-%dT%H:%M:%S.%f")
        except ValueError:
            return None

    coll.drop()

    with open(file, encoding='utf-8') as f:
        # La llamada csv.reader() crea un iterador sobre un fichero CSV
        reader = csv.reader(f, dialect='excel')

        # Se leen las columnas. Sus nombres se usarán para crear las diferentes columnas en la familia
        columns: list[str] = next(reader)

        # Las columnas que contienen 'Date' se interpretan como fechas
        func_to_cols: list[Callable[[str], Any]] = list(map(lambda c: to_date if 'date' in c.lower() else to_numeric, columns))

        for batch in batched(tqdm(reader, desc='Leyendo e insertando filas...'), 10000):
            docs: list[dict[str, str | datetime | float | int | None]] = []
            for row in batch:
                built_row: list[str | datetime | float | int | None] = [func(e) for (func,e) in zip(func_to_cols, row)]
                docs.append(dict(zip(columns, built_row)))
            coll.insert_many(docs)

        print("¡Hecho!")

In [15]:
csv_to_mongo('Posts.csv',db.posts)

Leyendo e insertando filas...: 0it [00:00, ?it/s]

¡Hecho!


In [16]:
csv_to_mongo('Users.csv',db.users)

Leyendo e insertando filas...: 0it [00:00, ?it/s]

¡Hecho!


In [17]:
csv_to_mongo('Votes.csv',db.votes)

Leyendo e insertando filas...: 0it [00:00, ?it/s]

¡Hecho!


In [18]:
csv_to_mongo('Comments.csv',db.comments)

Leyendo e insertando filas...: 0it [00:00, ?it/s]

¡Hecho!


In [19]:
csv_to_mongo('Tags.csv',db.tags)

Leyendo e insertando filas...: 0it [00:00, ?it/s]

¡Hecho!


In [20]:
posts: Collection[dict[str, Any]] = db.posts
posts

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'stackoverflow'), 'posts')

In [22]:
posts.count_documents({})

410346

## Framework de Agregación

Framework de agregación:
- Aquí está la referencia de las diferentes etapas por las que puede pasar un pipeline: https://www.mongodb.com/docs/manual/reference/operator/aggregation-pipeline/.
- Y aquí los distintos operadores que se permiten dentro de las etapas: https://docs.mongodb.com/manual/reference/operator/aggregation/.
- Y aquí incluso un libro completo con usos prácticos de ejecutar agregación: https://www.practical-mongodb-aggregations.com/.

A continuación un vídeo interesante:

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('VSX4a3h4SmQ',width=600)

Aqui tenemos un esquema con un ejemplo básico de agregación.

![](https://miro.medium.com/max/1060/1*2lDBxvZ8Cr3JYkoODTa0lQ.png)

## Algunos operadores de agregación



### `$match`

Este operador permite filtrar los documentos que queremos que pasen a la siguiente fase del pipeline definiendo una serie de condiciones sobre los campos de los mismos.

Vamos a filtrar aquellos documentos que tengan un `Score` igual o superior (`$gte`) a 150.

In [23]:
from pymongo.command_cursor import CommandCursor

respuestas: CommandCursor[dict[str, Any]] = posts.aggregate( [
        {'$match': { 'Score' : {'$gte': 150}}}
])
list(respuestas)

[{'_id': ObjectId('67c03f67932d153fdce87d12'),
  'Id': 40,
  'AcceptedAnswerId': '',
  'AnswerCount': '',
  'Body': '<p>Asumiendo que se está haciendo un <code>join</code> de columnas sin duplicados, lo cuál es un caso común:</p><br/><ul><br/><li><p>Un <strong>inner join</strong> de A y B entregará el resultado de la intersección de los conjuntos A y B. En otras palabras, la parte <em>interna</em> –intersección– en un diagrama de Venn.</p><br/></li><br/><li><p>Un <strong>full outer join</strong> entre A y B entregará el resultado de la unión de A y B. En otras palabras, la parte <em>externa</em> –unión– en un diagrama de Venn .</p><br/></li><br/></ul><br/><h1>Ejemplos:</h1><br/><p>Supongamos que tenemos dos tablas, con una sola columna cada una y los siguientes datos:</p><br/><pre><code>A    B<br/>-    -<br/>1    3<br/>2    4<br/>3    5<br/>4    6<br/></code></pre><br/><p>Nota que (1,2) solo se encuentran en A, (3,4) son comunes y (5,6) solamente se encuentran en B.</p><br/><h1>Inner J

### `$project`

El operador `$project`permite filtar qué campos de los documentos queremos usar en la siguientes fases de agregación.

Generalmente este operador suele combinarse con otros como `$find`, `$match` o `$lookup` en fases más avanzadas de la agregación.

En el siguiente ejemplo, realizamos un filtrado en donde solo nos quedamos con el campo `Id` de los posts.

In [24]:
respuestas: CommandCursor[dict[str, Any]] = db['posts'].aggregate( [
    {'$project' : { 'Id' : True }},
    {'$limit': 20} ])
list(respuestas)

[{'_id': ObjectId('67c03f67932d153fdce87cee'), 'Id': 1},
 {'_id': ObjectId('67c03f67932d153fdce87cef'), 'Id': 2},
 {'_id': ObjectId('67c03f67932d153fdce87cf0'), 'Id': 3},
 {'_id': ObjectId('67c03f67932d153fdce87cf1'), 'Id': 4},
 {'_id': ObjectId('67c03f67932d153fdce87cf2'), 'Id': 5},
 {'_id': ObjectId('67c03f67932d153fdce87cf3'), 'Id': 6},
 {'_id': ObjectId('67c03f67932d153fdce87cf4'), 'Id': 7},
 {'_id': ObjectId('67c03f67932d153fdce87cf5'), 'Id': 8},
 {'_id': ObjectId('67c03f67932d153fdce87cf6'), 'Id': 9},
 {'_id': ObjectId('67c03f67932d153fdce87cf7'), 'Id': 10},
 {'_id': ObjectId('67c03f67932d153fdce87cf8'), 'Id': 11},
 {'_id': ObjectId('67c03f67932d153fdce87cf9'), 'Id': 12},
 {'_id': ObjectId('67c03f67932d153fdce87cfa'), 'Id': 14},
 {'_id': ObjectId('67c03f67932d153fdce87cfb'), 'Id': 15},
 {'_id': ObjectId('67c03f67932d153fdce87cfc'), 'Id': 16},
 {'_id': ObjectId('67c03f67932d153fdce87cfd'), 'Id': 17},
 {'_id': ObjectId('67c03f67932d153fdce87cfe'), 'Id': 18},
 {'_id': ObjectId('67c0

### `$group`

La función `$group` se usa dentro del *pipeline* de agregación de documentos seguido por Mongo (`.aggregate`). Esta función admite dos parámetros diferentes:
* `_id`: El identificador por el que queremos agrupar los documentos.
* `campo`: Expresión mediante la cual queremos aggregar los documentos (*opcional*).

Vamos a contar el número de posts por `OwnerUserId` en la colección `posts`.

In [25]:
from pymongo.command_cursor import CommandCursor

users_count_scores: CommandCursor[dict[str, Any]] = db.posts.aggregate(
    [
        {
            "$group":{
                "_id": "$OwnerUserId",
                "count": {"$sum": 1}
            }
        },
        {
            "$limit": 10
        }
    ]
)
list(users_count_scores)

[{'_id': 48901, 'count': 1},
 {'_id': 56705, 'count': 123},
 {'_id': 314042, 'count': 2},
 {'_id': 342176, 'count': 1},
 {'_id': 214374, 'count': 1},
 {'_id': 2198, 'count': 1},
 {'_id': 154421, 'count': 2},
 {'_id': 326353, 'count': 2},
 {'_id': 139810, 'count': 1},
 {'_id': 19069, 'count': 4}]

Ahora vamos computar el `score` medio por usuario en base a todos sus posts.

In [26]:
users_avg_scores: CommandCursor[dict[str, Any]] = db.posts.aggregate(
    [{
        "$group":{
            "_id": "$OwnerUserId",
            "avg_score": {"$avg":'$Score'}
        }
        },
        { "$limit": 10 }
    ]
)
list (users_avg_scores)

[{'_id': 48901, 'avg_score': 1.0},
 {'_id': 56705, 'avg_score': 0.7154471544715447},
 {'_id': 314042, 'avg_score': 0.0},
 {'_id': 342176, 'avg_score': 0.0},
 {'_id': 214374, 'avg_score': 0.0},
 {'_id': 2198, 'avg_score': 2.0},
 {'_id': 154421, 'avg_score': 0.0},
 {'_id': 326353, 'avg_score': 1.5},
 {'_id': 139810, 'avg_score': 0.0},
 {'_id': 19069, 'avg_score': 0.25}]

### `$lookup`

El operador `$lookup` permite realizar búsquedas en otras colecciones. Podrían interpretarse como un *join* en el modelo relacional. Esta función admite cuatro parámetros diferentes:

*   `from`: especifica la colección en la misma base de datos con la que hacer la búsqueda (o el *join*).
*   `localField`: Especifica el campo de los documentos de entrada a usar para realizar una comparación de igualdad con el campo `foreignField` de los documentos de la colección `from`.  
*   `foreignField`: Especifica el campo de los documentos de la colección `from` a usar en la comparación de igualdad con el `localField`.
*   `as`: Especifica el nombre del nuevo campo array que se añadirá a los documentos de entrada. Este nuevo campo contiene los documentos coincidentes de la colección `from`.

En el ejemplo siguiente, en primer lugar seleccionamos los posts con un `Score` igual o mayor a 150 con `$match` para a continuación seleccionar los usuarios que han publicado dichos posts incluyéndolo en un nuevo campo llamado `owner`.

In [27]:
respuestas: CommandCursor[dict[str, Any]] = posts.aggregate( [
    {'$match': { 'Score' : {'$gte': 150}}},
    {'$lookup': {
        'from': "users",
        'localField': "OwnerUserId",
        'foreignField': "Id",
        'as': "owner"}
    }
])
list(respuestas)

[{'_id': ObjectId('67c03f67932d153fdce87d12'),
  'Id': 40,
  'AcceptedAnswerId': '',
  'AnswerCount': '',
  'Body': '<p>Asumiendo que se está haciendo un <code>join</code> de columnas sin duplicados, lo cuál es un caso común:</p><br/><ul><br/><li><p>Un <strong>inner join</strong> de A y B entregará el resultado de la intersección de los conjuntos A y B. En otras palabras, la parte <em>interna</em> –intersección– en un diagrama de Venn.</p><br/></li><br/><li><p>Un <strong>full outer join</strong> entre A y B entregará el resultado de la unión de A y B. En otras palabras, la parte <em>externa</em> –unión– en un diagrama de Venn .</p><br/></li><br/></ul><br/><h1>Ejemplos:</h1><br/><p>Supongamos que tenemos dos tablas, con una sola columna cada una y los siguientes datos:</p><br/><pre><code>A    B<br/>-    -<br/>1    3<br/>2    4<br/>3    5<br/>4    6<br/></code></pre><br/><p>Nota que (1,2) solo se encuentran en A, (3,4) son comunes y (5,6) solamente se encuentran en B.</p><br/><h1>Inner J

### `$arrayElemAt`

El `$lookup` genera un _array_ con todos los resultados. El operador `$arrayElementAt` accede al primer elemento.

In [28]:
respuestas: CommandCursor[dict[str, Any]] = db.posts.aggregate( [
    {'$match': { 'Score' : {'$gte': 150}}},
    {'$lookup': {
        'from': "users",
        'localField': "OwnerUserId",
        'foreignField': "Id",
        'as': "owner"}
    },
    { '$project' : {
        'Id' : True,
        'Score' : True,
        'username' : {'$arrayElemAt' : ['$owner.DisplayName', 0]},
        'owner.DisplayName' : True
      }}
    ])
list(respuestas)

[{'_id': ObjectId('67c03f67932d153fdce87d12'),
  'Id': 40,
  'Score': 206,
  'owner': [{'DisplayName': 'jachguate'}],
  'username': 'jachguate'},
 {'_id': ObjectId('67c03f67932d153fdce87d92'),
  'Id': 197,
  'Score': 176,
  'owner': [{'DisplayName': 'Peter Olson'}],
  'username': 'Peter Olson'},
 {'_id': ObjectId('67c03f67932d153fdce87d96'),
  'Id': 204,
  'Score': 195,
  'owner': [{'DisplayName': 'Konamiman'}],
  'username': 'Konamiman'},
 {'_id': ObjectId('67c03f69932d153fdce8b560'),
  'Id': 18232,
  'Score': 232,
  'owner': [{'DisplayName': 'Alvaro Montoro'}],
  'username': 'Alvaro Montoro'},
 {'_id': ObjectId('67c03f69932d153fdce8b561'),
  'Id': 18233,
  'Score': 219,
  'owner': [{'DisplayName': 'Alvaro Montoro'}],
  'username': 'Alvaro Montoro'},
 {'_id': ObjectId('67c03f6d932d153fdce97e17'),
  'Id': 87842,
  'Score': 173,
  'owner': [{'DisplayName': 'Alvaro Montoro'}],
  'username': 'Alvaro Montoro'}]

### `$unwind`

Este operador *desdobla* cada fila por cada elemento del array.

Ej: El siguiente código:

```python
db.inventario.insert_one({ "_id" : 1, "item" : "ABC1", "tallas": [ "S", "M", "L"] })
db.inventario.aggregate( [ { "$unwind" : "$tallas" } ] )
```

Devolverá:

```json
{ "_id" : 1, "item" : "ABC1", "tallas" : "S" }
{ "_id" : 1, "item" : "ABC1", "tallas" : "M" }
{ "_id" : 1, "item" : "ABC1", "tallas" : "L" }
```

En el ejemplo de procesamiento de posts que estamos llevando a cabo, como sabemos que el array `$owner` sólo contiene un elemento, sólo habrá una fila por fila original, pero sin el _array_. Finalmente se puede proyectar el campo que se quiera. En este caso el `ownerDisplayName` que es proyectado (renombrado) como `username`.

In [29]:
respuestas: CommandCursor[dict[str, Any]] = db.posts.aggregate( [
    { '$match': { 'Score' : {'$gte': 150}}},
    { '$lookup': {
        'from': "users",
        'localField': "OwnerUserId",
        'foreignField': "Id",
        'as': "owner"
      }
    },
    { '$unwind': '$owner'}, # Hace lo mismo que display pero sin tantos comandos
    { '$project' : {
          'username': '$owner.DisplayName'
      }
    }
    ])
list(respuestas)

[{'_id': ObjectId('67c03f67932d153fdce87d12'), 'username': 'jachguate'},
 {'_id': ObjectId('67c03f67932d153fdce87d92'), 'username': 'Peter Olson'},
 {'_id': ObjectId('67c03f67932d153fdce87d96'), 'username': 'Konamiman'},
 {'_id': ObjectId('67c03f69932d153fdce8b560'), 'username': 'Alvaro Montoro'},
 {'_id': ObjectId('67c03f69932d153fdce8b561'), 'username': 'Alvaro Montoro'},
 {'_id': ObjectId('67c03f6d932d153fdce97e17'), 'username': 'Alvaro Montoro'}]

### `$push`

Devuelve un array con *todos* los valores que resultan de aplicar una determinada expresión a los documentos que forman parte del pipeline.

Obtener el listado con el identificador de `Posts` (`Id`) asociados a cada valor de `Score`.

In [32]:
posts_by_score: CommandCursor[dict[str, Any]] = db.posts.aggregate([
    { '$limit': 2000 },
    {
        '$group':{
            '_id': '$Score',
            'posts':{
                '$push': {
                    'post': '$Id'
                }
            }
        }
    }

])
list(posts_by_score)

[{'_id': 22,
  'posts': [{'post': 47},
   {'post': 112},
   {'post': 130},
   {'post': 267},
   {'post': 350},
   {'post': 821},
   {'post': 972},
   {'post': 1366},
   {'post': 1371},
   {'post': 1958}]},
 {'_id': 50, 'posts': [{'post': 607}]},
 {'_id': 10,
  'posts': [{'post': 16},
   {'post': 83},
   {'post': 84},
   {'post': 110},
   {'post': 169},
   {'post': 221},
   {'post': 250},
   {'post': 255},
   {'post': 269},
   {'post': 285},
   {'post': 287},
   {'post': 313},
   {'post': 316},
   {'post': 318},
   {'post': 365},
   {'post': 393},
   {'post': 447},
   {'post': 457},
   {'post': 512},
   {'post': 540},
   {'post': 772},
   {'post': 841},
   {'post': 854},
   {'post': 899},
   {'post': 944},
   {'post': 961},
   {'post': 1018},
   {'post': 1134},
   {'post': 1144},
   {'post': 1284},
   {'post': 1740},
   {'post': 1860},
   {'post': 1873},
   {'post': 1919},
   {'post': 2019},
   {'post': 2194}]},
 {'_id': 25,
  'posts': [{'post': 11},
   {'post': 228},
   {'post': 363},


### `$addToSet`

 Devuelve un array de todos los valores únicos que resultan de aplicar una expresión a cada documento de un grupo.


Vamos a obtener el listado de `Tags` asociadas a cada tipo de licencia  `ContentLicense`.

In [30]:
license_and_tags: CommandCursor[dict[str, Any]] = db.posts.aggregate([
    { "$match" : {"PostTypeId": 1}},
    {"$limit": 1000},

    {'$group':{
            '_id':'$ContentLicense',
            'AllTags': { '$addToSet': "$Tags" }
        }
    }
])
list(license_and_tags)

[{'_id': 'CC BY-SA 4.0',
  'AllTags': ['<base-de-datos><cálculo-relacional>',
   '<python><python-3.x>',
   '<python><django><python-2.7><utf-8><codificación>',
   '<mysql><sql><join>',
   '<python><bootstrap><django><python-3.x><select2>',
   '<git><git-merge>',
   '<java><ordenamiento><lista>',
   '<c#><ofuscación>',
   '<c#><reflexión>',
   '<java><android><android-intent><android-asynctask>',
   '<c#><.net><entity-framework><asp.net-core><model-view-controller>',
   '<javascript><jquery><asp.net-mvc>',
   '<nodejs><loopback>',
   '<c#><sql-server><windows-forms>',
   '<javascript><jquery><asincronismo>',
   '<c#><asp.net><excel>',
   '<java><java-swing>',
   '<xml>',
   '<raku>',
   '<javascript><hapijs>',
   '<asp.net><asp.net-mvc><librería><bower>',
   '<.net><asp.net-web-api><asp.net>',
   '<python><djangorestframework><django-models>',
   '<python><python-2.7>',
   '<javascript><validación><email><regex>',
   '<bash><verilog>',
   '<python><pip><pillow>',
   '<archivo><fish><ex

### `$out`

Este comando permite volcar el resultado de un pipeline de agregación en una nueva colección en la base de datos.

In [33]:
respuestas: CommandCursor[dict[str, Any]] = db.posts.aggregate( [
        { '$match': { 'Score' : {'$gte': 40}}},
        { '$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"
          }
        },
        { '$unwind': '$owner'},
        { '$project' : {
             'username': '$owner.DisplayName'
          }
        },
        {'$out': "stackoverflow_users"}
      ])

Un documento de ejemplo de la colección generada `stackoverflow_users`:

In [34]:
pp(db.stackoverflow_users.find_one())

{'_id': ObjectId('67c03f67932d153fdce87cee'), 'username': 'Konamiman'}


In [35]:
db.stackoverflow_users.count_documents({})

122

## Ejemplos básicos de agregación

### Ejemplo 1: Asociación usuarios con Tags

Con Agregación, vamos a construir una colección que asocia un usuario con los tags que ha usado en todas sus preguntas.

*Por cuestiones de rendimiento vamos a limitar la agregación a 500 elementos.*

In [36]:
user_tags: CommandCursor[dict[str, Any]] = db.posts.aggregate([
        { "$match" : {"PostTypeId": 1}},

        {"$limit": 500},

        { '$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"
          }
        },

        {'$project':{
            'Tags': True,
            'userid' : {'$arrayElemAt' : ['$owner.Id', 0]},
        }},

        {'$group':{
            '_id':'$userid',
            'AllTags': { '$addToSet': "$Tags" }
        }}
]
)

user_tags_lst = list(user_tags)
user_tags_lst

[{'_id': 305,
  'AllTags': ['<android><android-softkeyboard>',
   '<android><android-layout><unidades-de-medición>']},
 {'_id': 25,
  'AllTags': ['<javascript>',
   '<git>',
   '<c++><c++11>',
   '<licencias><código-abierto>',
   '<c++><array>',
   '<c#>']},
 {'_id': 1809, 'AllTags': ['<java><excel><apache-poi>', '<java><apache-poi>']},
 {'_id': 21,
  'AllTags': ['<javascript><jquery>',
   '<android><arduino><socket><delphi><tcpip>',
   '<mysql><datetime><timestamp>',
   '<mysql><sql><join>',
   '<delphi>',
   '<sql><oracle><oracle-10g>']},
 {'_id': 2325, 'AllTags': ['<c#>']},
 {'_id': 2113, 'AllTags': ['<javascript>']},
 {'_id': 400,
  'AllTags': ['<javascript><java><jquery>',
   '<java><thymeleaf>',
   '<javascript><jquery><thymeleaf>',
   '<java><jpa>',
   '<java><alfresco>',
   '<javascript><jquery><asincronismo>']},
 {'_id': 870, 'AllTags': ['<hilos>', '<python><python-2.7>']},
 {'_id': 1076, 'AllTags': ['<select2><yii><yiibooster><cache>']},
 {'_id': 1948, 'AllTags': ['<jsf><prim

### Ejemplo 2: Asociación Tags con usuarios

Ahora, dado un ID de tag, qué usuarios han hecho preguntas de ese tag.

*De nuevo limitamos la búsqueda a 500 documentos*

In [37]:
tags_users: CommandCursor[dict[str, Any]] = db.posts.aggregate( [
        { "$match" : {"PostTypeId": 1}},

        {"$limit": 500},

        { '$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"
          }
        },

        {'$project':{
            'Tags': True,
            'userid' : {'$arrayElemAt' : ['$owner.Id', 0]},
        }},

        {'$group':{
            '_id':'$Tags',
            'AllUsers': { '$addToSet': "$userid" }
        }}
    ])

tags_users_lst = list(tags_users)
tags_users_lst

[{'_id': '<php><timestamp>', 'AllUsers': [132]},
 {'_id': '<python><django><django-models><users-roles>', 'AllUsers': [1005]},
 {'_id': '<django><django-models>', 'AllUsers': [1005]},
 {'_id': '<sequelize.js><pruebas-unitarias>', 'AllUsers': [345]},
 {'_id': '<html><xpath>', 'AllUsers': [729]},
 {'_id': '<c#><uwp>', 'AllUsers': [2230]},
 {'_id': '<ios><xcode><swift>', 'AllUsers': [1527]},
 {'_id': '<git><git-merge>', 'AllUsers': [555]},
 {'_id': '<ios><jquery><cordova>', 'AllUsers': [239]},
 {'_id': '<c#><xaml><windows-forms><wpf>', 'AllUsers': [2238]},
 {'_id': '<google-maps><angularjs><jasmine><karma-jasmine>', 'AllUsers': [23]},
 {'_id': '<javascript><jquery>',
  'AllUsers': [2156, 21, 2238, 2224, 729, 1839]},
 {'_id': '<java><clase>', 'AllUsers': [301]},
 {'_id': '<delphi>', 'AllUsers': [21]},
 {'_id': '<ios><xcode><objective-c>', 'AllUsers': [1499]},
 {'_id': '<css>', 'AllUsers': [342]},
 {'_id': '<javascript><google-apps-script><google-hoja-de-cálculo><google-forms><google-docume

## Ejercicios

### EJ1: Obtener el resultado de documentos *verbosos* en donde se indique el texto `Body` de una pregunta así como el nombre del usuario que la formuló (`DisplayName`)

In [40]:
verbose_posts: CommandCursor[dict[str, Any]] = db.posts.aggregate([
        { "$match" : {"PostTypeId": 1}}, # 1 es el id de tipo pregunta

        {"$limit": 10},

        { '$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"
          }
        },

        {'$project':{
            'Body': True,
            'userid' : {'$arrayElemAt' : ['$owner.DisplayName', 0]},
        }}

])

list(verbose_posts)

[{'_id': ObjectId('67c03f67932d153fdce87cee'),
  'Body': '<p>Estoy creando un servicio usando <em>ASP.NET WebApi</em>. Quiero añadir soporte para la negociación del tipo de contenido basado en extensiones en el <em>URI</em>, así que he añadido lo siguiente al código de inicialización del servicio:</p><br/><br/><pre><code>public static class WebApiConfig<br/>{<br/>  public static void Register(HttpConfiguration config)<br/>  {<br/>    config.Formatters.JsonFormatter.AddUriPathExtensionMapping("json", "application/json");<br/>    config.Formatters.XmlFormatter.AddUriPathExtensionMapping("xml", "application/xml");<br/>  }<br/>}<br/></code></pre><br/><br/><p>Para que esto funcione necesito crear dos rutas para cada acción del controlador (estoy usando exclusivamente enrutamiento basado en atributos):</p><br/><br/><pre><code>[Route("item/{id}/details")]<br/>[Route("item/{id}/details.{ext}")]<br/>[HttpGet]<br/>public ItemDetail[] GetItemDetails(int id)<br/>{<br/>  return itemsService.GetItem

In [42]:
# Para que no salga el userID

verbose_posts: CommandCursor[dict[str, Any]] = db.posts.aggregate([
        { "$match" : {"PostTypeId": 1}}, # 1 es el id de tipo pregunta

        {"$limit": 10},

        { '$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"
          }
        },

        {'$project':{
            '_id': False,
            'Body': True,
            'userid' : {'$arrayElemAt' : ['$owner.DisplayName', 0]},
        }}

])

list(verbose_posts)

[{'Body': '<p>Estoy creando un servicio usando <em>ASP.NET WebApi</em>. Quiero añadir soporte para la negociación del tipo de contenido basado en extensiones en el <em>URI</em>, así que he añadido lo siguiente al código de inicialización del servicio:</p><br/><br/><pre><code>public static class WebApiConfig<br/>{<br/>  public static void Register(HttpConfiguration config)<br/>  {<br/>    config.Formatters.JsonFormatter.AddUriPathExtensionMapping("json", "application/json");<br/>    config.Formatters.XmlFormatter.AddUriPathExtensionMapping("xml", "application/xml");<br/>  }<br/>}<br/></code></pre><br/><br/><p>Para que esto funcione necesito crear dos rutas para cada acción del controlador (estoy usando exclusivamente enrutamiento basado en atributos):</p><br/><br/><pre><code>[Route("item/{id}/details")]<br/>[Route("item/{id}/details.{ext}")]<br/>[HttpGet]<br/>public ItemDetail[] GetItemDetails(int id)<br/>{<br/>  return itemsService.GetItemDetails(id);<br/>}<br/><br/>[Route("item/{name}

### EJ2: Obtener las respuestas escritas en Enero de 2022 con un `Score` superior a 3

In [44]:
start = datetime(2022, 1, 1, 0, 0, 0)
end = datetime(2022, 1, 31)

responses_jan22: CommandCursor[dict[str, Any]] = db.posts.aggregate([
        { "$match" : {
            "PostTypeId": 2, # 2 es el id de tipo respuesta
            'Score' : {'$gt': 3},
            'CreationDate': {'$gte':start, '$lte': end} }},
])

list(responses_jan22)

[{'_id': ObjectId('67c03f89932d153fdcedb3f1'),
  'Id': 507164,
  'AcceptedAnswerId': '',
  'AnswerCount': '',
  'Body': '<p>Buen día! esto se puede solucionar de varias maneras, sin el uso de paquetes</p><br/><p><div class="snippet" data-lang="js" data-hide="false" data-console="true" data-babel="false"><br/><div class="snippet-code"><br/><pre class="snippet-code-js lang-js prettyprint-override"><code>Date.prototype.addDays = function(days) {<br/>    var date = new Date(this.valueOf());<br/>    date.setDate(date.getDate() + days);<br/>    return date;<br/>}<br/><br/>var date = new Date();<br/><br/>console.log(date.addDays(1));</code></pre><br/></div><br/></div><br/></p><br/><p>O utilizando paquetes como DayJs que facilitan la manipulación de fechas:</p><br/><ul><br/><li>Instalación de paquete: <a href="https://www.npmjs.com/package/dayjs" rel="nofollow noreferrer">https://www.npmjs.com/package/dayjs</a></li><br/><li>Método que vas a necesitar: <a href="https://day.js.org/docs/en/manipu

In [ ]:
print("Eso es todo amigos!")